In [1]:
#link data from kaggle 
#https://www.kaggle.com/uciml/breast-cancer-wisconsin-data 

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
import io
from google.colab import files
%matplotlib inline

In [3]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

Saving data.csv to data.csv


In [4]:
df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,1.1760,1.2560,7.673,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,0.7655,2.4630,5.203,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,0.4564,1.0750,3.425,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,0.7260,1.5950,5.772,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [5]:
# Chuẩn đoán các giá trị trong cột diagnosis -> hình dung số lượng giá trị của các cột 'diagnosis' để dễ hiểu hơn
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [6]:
#Tìm xem trong data các cột có giá trị null
df.isnull().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [7]:
df.drop(['Unnamed: 32','id'],axis=1,inplace=True)

In [8]:
# Trong dataset có 2 biến : biến độc lập và phụ thuộc. Nên chúng ta chia ra làm 2
# Biến độc lập
x = df.drop('diagnosis',axis=1)
#Biến phụ thuộc
y = df.diagnosis

In [9]:
# Do y có chứa data phân loại nên sẽ convert data sang binary
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y = lb.fit_transform(y)

In [10]:
#Chuẩn hoá dữ liệu
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=40)

In [11]:
#Scaling dữ liệu
#importing StandardScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xtrain = sc.fit_transform(xtrain)
xtest = sc.transform(xtest)

# **ANN**

In [12]:
#import các module từ keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

In [13]:
#Tạo model
classifier = Sequential()

In [14]:
# Thêm các input layer,first hidden layer
classifier.add(Dense(units=16, activation='relu', input_dim=30,kernel_initializer="uniform"))
# add dropout -> ngăn overfitting
classifier.add(Dropout(rate=0.1))

In [15]:
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

In [16]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                496       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


In [17]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
classifier.fit(xtrain, ytrain, batch_size=100, epochs=150)

Epoch 1/150
4/4 [==============================] - 1s 4ms/step - loss: 0.6921 - accuracy: 0.6106
Epoch 2/150
4/4 [==============================] - 0s 5ms/step - loss: 0.6866 - accuracy: 0.8819
Epoch 3/150
4/4 [==============================] - 0s 4ms/step - loss: 0.6804 - accuracy: 0.8920
Epoch 4/150
4/4 [==============================] - 0s 3ms/step - loss: 0.6730 - accuracy: 0.9020
Epoch 5/150
4/4 [==============================] - 0s 6ms/step - loss: 0.6634 - accuracy: 0.9095
Epoch 6/150
4/4 [==============================] - 0s 5ms/step - loss: 0.6521 - accuracy: 0.9070
Epoch 7/150
4/4 [==============================] - 0s 4ms/step - loss: 0.6389 - accuracy: 0.9070
Epoch 8/150
4/4 [==============================] - 0s 5ms/step - loss: 0.6241 - accuracy: 0.9146
Epoch 9/150
4/4 [==============================] - 0s 5ms/step - loss: 0.6066 - accuracy: 0.9146
Epoch 10/150
4/4 [==============================] - 0s 5ms/step - loss: 0.5882 - accuracy: 0.9196
Epoch 11/150
4/4 [===========

# Clustering, Kmeans, Kmedoids

In [19]:
kmeans = KMeans(n_clusters=12)
model = kmeans.fit(xtrain)
print("model\n", model)

model
 KMeans(n_clusters=12)


In [20]:
centers = model.cluster_centers_
centers

array([[-4.10618966e-02, -4.51840985e-02, -2.27457965e-02,
        -1.45153483e-01,  3.48287262e-01,  3.51269140e-01,
         4.34880984e-02,  1.02174526e-01,  1.19328209e-01,
         1.96001446e-01, -3.80450831e-01, -4.75522973e-01,
        -3.36042240e-01, -3.19790493e-01, -3.96804899e-01,
         5.60995953e-02, -4.97914921e-02,  5.79771668e-03,
        -4.19628117e-01, -8.79501867e-02, -4.74604279e-02,
         1.75183913e-02, -1.39342888e-02, -1.58882393e-01,
         4.17640257e-01,  5.39554229e-01,  3.06360656e-01,
         3.83767435e-01,  2.44300605e-01,  4.90056753e-01],
       [-3.21565672e-01,  7.51625661e-01, -3.72687999e-01,
        -3.50727646e-01, -8.77389363e-01, -9.41792853e-01,
        -7.76855091e-01, -7.23278300e-01, -6.85046882e-01,
        -7.49841580e-01, -2.40785577e-01,  5.61976680e-01,
        -3.07119105e-01, -2.83647153e-01, -4.37852560e-01,
        -7.64675092e-01, -5.75586514e-01, -6.33294579e-01,
        -2.62059158e-01, -5.79403359e-01, -3.64077227e-

In [21]:
!pip install scikit-learn-extra

     |████████████████████████████████| 1.7 MB 7.7 MB/s 


In [22]:
from sklearn_extra.cluster import KMedoids
kmedoids = KMedoids(n_clusters=12)
kmedoids.fit(xtrain)

KMedoids(n_clusters=12)

In [23]:
centroid = kmeans.cluster_centers_
labels = kmeans.labels_

print(centroid)
print(labels)

[[-4.10618966e-02 -4.51840985e-02 -2.27457965e-02 -1.45153483e-01
   3.48287262e-01  3.51269140e-01  4.34880984e-02  1.02174526e-01
   1.19328209e-01  1.96001446e-01 -3.80450831e-01 -4.75522973e-01
  -3.36042240e-01 -3.19790493e-01 -3.96804899e-01  5.60995953e-02
  -4.97914921e-02  5.79771668e-03 -4.19628117e-01 -8.79501867e-02
  -4.74604279e-02  1.75183913e-02 -1.39342888e-02 -1.58882393e-01
   4.17640257e-01  5.39554229e-01  3.06360656e-01  3.83767435e-01
   2.44300605e-01  4.90056753e-01]
 [-3.21565672e-01  7.51625661e-01 -3.72687999e-01 -3.50727646e-01
  -8.77389363e-01 -9.41792853e-01 -7.76855091e-01 -7.23278300e-01
  -6.85046882e-01 -7.49841580e-01 -2.40785577e-01  5.61976680e-01
  -3.07119105e-01 -2.83647153e-01 -4.37852560e-01 -7.64675092e-01
  -5.75586514e-01 -6.33294579e-01 -2.62059158e-01 -5.79403359e-01
  -3.64077227e-01  6.61921510e-01 -4.23880865e-01 -3.82838742e-01
  -9.62671560e-01 -8.98759609e-01 -8.15947344e-01 -8.18792836e-01
  -7.13787840e-01 -8.84180684e-01]
 [ 1.8

# SVM


In [24]:
from sklearn import svm
clf1 = svm.SVC(kernel='linear',C=1,gamma='auto') # Linear Kernel
clf1.fit(xtrain, ytrain)

y_pred_1= clf1.predict(xtest)

In [25]:
clf2 = svm.SVC(kernel='linear',C=2,gamma='auto') # Linear Kernel
clf2.fit(xtrain, ytrain)

y_pred_2= clf2.predict(xtest)

In [26]:
clf3 = svm.SVC(kernel='linear',C=3,gamma='auto') # Linear Kernel
clf3.fit(xtrain, ytrain)

y_pred_3= clf3.predict(xtest)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(ytest,y_pred_1)) # C= 1 
print(classification_report(ytest,y_pred_2)) # C= 2 
print(classification_report(ytest,y_pred_3)) # C= 3 

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       115
           1       0.96      0.96      0.96        56

    accuracy                           0.98       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171

              precision    recall  f1-score   support

           0       0.98      0.97      0.98       115
           1       0.95      0.96      0.96        56

    accuracy                           0.97       171
   macro avg       0.96      0.97      0.97       171
weighted avg       0.97      0.97      0.97       171

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       115
           1       0.96      0.96      0.96        56

    accuracy                           0.98       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



In [28]:
from sklearn import svm
from sklearn.model_selection import cross_val_score
svm = svm.SVC()
scores = cross_val_score(svm, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()
print("The accuracy is %s" % round(scores,3))

The accuracy is 0.972


# LR

In [29]:
# Import các model sklearn
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
# Tạo model:
lr = LogisticRegression()

scores = cross_val_score(lr, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()

print("The accuracy is %s" % round(scores,3))

The accuracy is 0.97


# kNN

In [30]:
# Import các model sklearn
from sklearn.neighbors import KNeighborsClassifier
# Tạo model:

knn = KNeighborsClassifier()

scores = cross_val_score(knn, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()

print("The accuracy is %s" % round(scores,3))

The accuracy is 0.962


# Perceptron

In [31]:
# Import các model sklearn
from sklearn.linear_model import Perceptron
# Tạo model:
pct = Perceptron()

scores = cross_val_score(pct, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()

print("The accuracy is %s" % round(scores,3))

The accuracy is 0.957


# Random Forest

In [32]:
# Import các model sklearn
from sklearn.ensemble import RandomForestClassifier
# Tạo model:
rf = RandomForestClassifier()

scores = cross_val_score(rf, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()

print("The accuracy is %s" % round(scores,3))

The accuracy is 0.957


#Adaboost

In [33]:
# Import
from sklearn.ensemble import AdaBoostClassifier

ada = RandomForestClassifier()
scores = cross_val_score(ada, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()

print("The accuracy is %s" % round(scores,3))

The accuracy is 0.967


#Gradient Boost

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
Gradboot = GradientBoostingClassifier(n_estimators=50)

scores = cross_val_score(Gradboot, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()
print("The accuracy is %s" % round(scores,3))

The accuracy is 0.952


# Naive Bayes

In [37]:
# Import các model sklearn
from sklearn.naive_bayes import GaussianNB
# Tạo model:
nb = GaussianNB()

scores = cross_val_score(nb, xtrain, ytrain, scoring='accuracy' ,cv=10).mean()

print("The accuracy is %s" % round(scores,3))

The accuracy is 0.937


# Thử trên test data


**Gradient Boost**

In [48]:
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.metrics as metrics

gra = GradientBoostingClassifier(n_estimators=50)

gra = gra.fit(xtrain, ytrain)

predicted = gra.predict(xtest)

acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.977


**Adaboost**

In [46]:
from sklearn.ensemble import AdaBoostClassifier
import sklearn.metrics as metrics
# Initiating the model:
ada = AdaBoostClassifier(n_estimators=18)

ada = ada.fit(xtrain, ytrain)

predicted = ada.predict(xtest)

acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.953


**RANDOM FOREST**

In [38]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
# Initiating the model:
rf = RandomForestClassifier(n_estimators=18)

rf = rf.fit(xtrain, ytrain)

predicted = rf.predict(xtest)

acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.965


**Naive Bayes**

In [39]:
from sklearn.naive_bayes import GaussianNB

# Initiating the model:
nb = GaussianNB()

nb = nb.fit(xtrain, ytrain)

predicted = nb.predict(xtest)

acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.965


**Perceptron**

In [40]:
# Import các model sklearn
from sklearn.linear_model import Perceptron
# Tạo model:
pct = Perceptron()
pct = pct.fit(xtrain, ytrain)
predicted = pct.predict(xtest)
acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.965


**kNN**

In [41]:
# Import các model sklearn
from sklearn.neighbors import KNeighborsClassifier
# Tạo model:

knn = KNeighborsClassifier()

knn = knn.fit(xtrain, ytrain)
predicted = knn.predict(xtest)
acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.971


**LR**

In [42]:
# Import các model sklearn
from sklearn.linear_model import LogisticRegression
# Tạo model:
lr = LogisticRegression()

lr = lr.fit(xtrain, ytrain)
predicted = lr.predict(xtest)
acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.977


**SVM**

In [43]:
from sklearn import svm

svm = svm.SVC()
svm = svm.fit(xtrain, ytrain)
predicted = svm.predict(xtest)
acc_test = metrics.accuracy_score(ytest, predicted)

print ('The accuracy on test data is %s' % (round(acc_test,3)))

The accuracy on test data is 0.988
